# 0.0 IMPORTS

In [31]:
import pandas as pd
import tiktoken
import torch

from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm



pd.set_option("display.max_colwidth", 200)

## 0.1 Helper Functions

In [2]:
summarizer = pipeline(
    task="summarization",
    model="facebook/bart-large-cnn"
)

def chunk_text(
    text: str,
    chunk_size: int = 1000,
    overlap: int = 150,
    encoding_name: str = "cl100k_base"
):
    enc = tiktoken.get_encoding(encoding_name)
    tokens = enc.encode(text)

    chunks = []
    start = 0

    while start < len(tokens):
        end = start + chunk_size
        chunk_tokens = tokens[start:end]
        chunk_text = enc.decode(chunk_tokens)
        chunks.append(chunk_text)

        start += chunk_size - overlap

    return chunks

def chunk_text_by_tokens(text, max_tokens=1000):
    tokens = enc.encode(text)
    
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk_tokens = tokens[i:i + max_tokens]
        chunk_text = enc.decode(chunk_tokens)
        chunks.append(chunk_text)
        
    return chunks


def summarize_text(chunk_text, max_length=150, min_length=60):
    result = summarizer(
        chunk_text,
        max_length=max_length,
        min_length=min_length,
        do_sample=False
    )
    return result[0]["summary_text"]


/home/tiago/.pyenv/versions/llm-docs/lib/python3.10/site-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## 0.2 Loading Data

In [3]:
df_raw = pd.read_csv( "../data/raw/train.csv")

df_raw.head()

,paper_id,text,summary
0,0,"## FROM SOVEREIGNTY TO EXTRATERRITORIAL CONSCIOUSNESS\n\n\nSince 1997, the concept of extraterritoriality has been configured in the political tension between Hong Kong and Beijing. From the persp...","In this article, Victor Fan argues that analysing contemporary Hong Kong cinema requires active rewriting of established postcolonial theories by taking into account the specific mode of colonisat..."
1,1,"## 1. Introduction\n\n\nAn Electronic Health Record (EHR) System is the digitized version of a patient's medical chart record containing medical history, diagnoses, treatment plans, immuniz...",Problem definition: Physicians spend more than 5 hours a day working on Electronic Health Record (EHR) systems and more than an hour doing EHR tasks after the end of the workday. Numerous studies ...
2,2,"## Introduction\n\n\nTranslation plays an important role in our life. Due to the proliferation of technology, globalization and many other reasons, the need for translation has increa...",Literary translation is one of the most challenging fields of translation because literary texts are rich in figurative language such as collocations and metaphors. This critical analysis discusse...
3,3,"## 1 Problem Setup\n\n\nRecent political science scholarship has questioned the ability to evaluate presidential election forecasts [1]. On its face, this argument is plausible: presidential elect...",There is a long-running debate on evaluating forecasts of rare events such as presidential elections. Recent scholarship has argued that forecasts cannot be evaluated on practical timelines and th...
4,4,"## INTRODUCTION\n\n\nThis article investigates the contemporary Japanese phenomenon that values youth over maturity, resulting in measures taken to postpone the latter. Throughout Japan,...","Recently, ‘bimajyo’ (美魔女) came into focus in Japan. Bimajyo are women 35 and above who look much younger than their actual ages. They put on nice make-up, their hair is nicely arranged, and they a..."


# 1.0 DESCRIÇÃO DOS DADOS

In [4]:
df1 = df_raw.copy()

In [5]:
df1.columns

Index(['paper_id', 'text', 'summary'], dtype='object')

## 1.1 Data Dimensions

In [6]:
print(f'Numero de Linhas: {df1.shape[0]}')
print(f'Numero de Colunas: {df1.shape[1]}')

Numero de Linhas: 1000
Numero de Colunas: 3


## 1.2 Data Types

In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   paper_id  1000 non-null   int64 
 1   text      1000 non-null   object
 2   summary   1000 non-null   object
dtypes: int64(1), object(2)
memory usage: 23.6+ KB


## 1.3 Check NA

In [8]:
df1.isna().sum()

paper_id    0
text        0
summary     0
dtype: int64

# 2.0 ANALISE EXPLORATORIA DE DADOS

In [9]:
df2 = df1.copy()

## 2.1 Analise Texto

In [10]:
df2['text'].iloc[0]

"## FROM SOVEREIGNTY TO EXTRATERRITORIAL CONSCIOUSNESS\n\n\nSince 1997, the concept of extraterritoriality has been configured in the political tension between Hong Kong and Beijing. From the perspective of the Central Government, it is fundamental for the people of China to shijian zhuquan instantiate its sovereignty over Hong Kong. But while most Hong / Kong residents insist on interpreting this concept in terms of the Euro-American notion of selfdetermination  (zizhu / making decisions  for  oneself),  the  Beijing  government  believes  that  the Hong Kong legislature must make decisions in conformation to the larger will of the people, which the Party represents, a concept taken from the writings of Lenin and Stalin (Gao 2010: 26-30). This tension is crystalised in the long debate about Article 23 of the Hong Kong Basic Law, which requires  the  SAR  to  'enact  laws  on  its  own  to  prohibit  any  act  of  treason,  secession,  sedition, subversion  against  the  Central  Peopl

In [11]:
df2['text'].iloc[1]

'## 1. Introduction\n\n\nAn Electronic Health Record (EHR) System is the digitized version of a patient\'s medical chart record  containing  medical  history,  diagnoses,  treatment  plans,  immunization  records,  and  test  results. 1 EHRs reduce diagnostic errors and patient safety concerns (Graber et al. 2017, Hydari et al. 2019). EHRs also improve coordination and integration of care by providing real-time data at the point of care, efficient transfer of information across settings, and physician decision-support (Rathert et al. 2019). As of 2021, 89% of office-based physicians in the US had adopted an EHR system (Office of the National Coordinator for Health Information Technology 2021). In a 2018 survey of 500 primary care physicians in the US, 63% of physicians agreed that EHRs had led to improved care. However, 71% of physicians also said EHRs significantly contribute to physician burnout (Stanford Medicine 2018).\n\nSinsky et al. (2016) found that outpatient clinicians spend 

In [12]:
# Quantidade de caracteres
df2['text_length'] = df2['text'].str.len()

df2['text_length'].describe()

count      1000.000000
mean      42050.269000
std       21412.146068
min       10145.000000
25%       28381.500000
50%       38962.000000
75%       51263.250000
max      196911.000000
Name: text_length, dtype: float64

In [13]:
# Quantidade de palavras
df2['word_count'] = df2['text'].str.findall(r'\w+').str.len()

df2['word_count'].describe()

count     1000.000000
mean      6398.120000
std       3310.587097
min        996.000000
25%       4278.500000
50%       5900.000000
75%       7825.000000
max      30107.000000
Name: word_count, dtype: float64

# 3.0 LIMPEZA DOS DADOS

In [14]:
df3 = df2.copy()

In [15]:
## Remoção de espaços extras e quebras de linhas

df3['text'] = (
    df3['text'].str.replace(r'\s+', ' ', regex=True)
               .str.strip()
)

## Remoção Caracteres invisíveis / estranhos
df3['text'] = (
    df3['text'].str.replace('\u00a0', ' ')
               .str.replace('\ufeff', '')
)

## Padronização haspas e Hífens
df3['text'] = (
    df3['text']
    .str.replace('“', '"')
    .str.replace('”', '"')
    .str.replace("’", "'")
    .str.replace("–", "-")
)
df3.head()

,paper_id,text,summary,text_length,word_count
0,0,"## FROM SOVEREIGNTY TO EXTRATERRITORIAL CONSCIOUSNESS Since 1997, the concept of extraterritoriality has been configured in the political tension between Hong Kong and Beijing. From the perspectiv...","In this article, Victor Fan argues that analysing contemporary Hong Kong cinema requires active rewriting of established postcolonial theories by taking into account the specific mode of colonisat...",33416,5331
1,1,"## 1. Introduction An Electronic Health Record (EHR) System is the digitized version of a patient's medical chart record containing medical history, diagnoses, treatment plans, immunization record...",Problem definition: Physicians spend more than 5 hours a day working on Electronic Health Record (EHR) systems and more than an hour doing EHR tasks after the end of the workday. Numerous studies ...,80639,12639
2,2,"## Introduction Translation plays an important role in our life. Due to the proliferation of technology, globalization and many other reasons, the need for translation has increased widely. There ...",Literary translation is one of the most challenging fields of translation because literary texts are rich in figurative language such as collocations and metaphors. This critical analysis discusse...,24014,3583
3,3,"## 1 Problem Setup Recent political science scholarship has questioned the ability to evaluate presidential election forecasts [1]. On its face, this argument is plausible: presidential elections ...",There is a long-running debate on evaluating forecasts of rare events such as presidential elections. Recent scholarship has argued that forecasts cannot be evaluated on practical timelines and th...,16296,2638
4,4,"## INTRODUCTION This article investigates the contemporary Japanese phenomenon that values youth over maturity, resulting in measures taken to postpone the latter. Throughout Japan, one can find m...","Recently, ‘bimajyo’ (美魔女) came into focus in Japan. Bimajyo are women 35 and above who look much younger than their actual ages. They put on nice make-up, their hair is nicely arranged, and they a...",43641,7297


# 4.0 TOKENS

In [16]:
df4 = df3.copy()

In [17]:
enc = tiktoken.get_encoding("cl100k_base")
df4['n_tokens'] = df4['text'].apply(lambda x: len(enc.encode(x)))
df4['n_tokens'].describe()

count     1000.000000
mean      8401.224000
std       4636.775045
min       1540.000000
25%       5484.000000
50%       7773.500000
75%      10161.750000
max      57461.000000
Name: n_tokens, dtype: float64

In [18]:
sample_text = df4.loc[1, 'text']

chunks = chunk_text(sample_text)

len(chunks)

22

In [19]:
chunks[0][:500]

"## 1. Introduction An Electronic Health Record (EHR) System is the digitized version of a patient's medical chart record containing medical history, diagnoses, treatment plans, immunization records, and test results. 1 EHRs reduce diagnostic errors and patient safety concerns (Graber et al. 2017, Hydari et al. 2019). EHRs also improve coordination and integration of care by providing real-time data at the point of care, efficient transfer of information across settings, and physician decision-su"

In [20]:
chunks[-1][:500]

' 6 show that prework and postwork can reduce after-hours work. Therefore, for at least small increases in idle time, the makespan for the physician reduces due to the reduction in afterhours work. ## 7.3 Limitations Our study has some limitations. First, our analysis is from a primary care setting. Physicians in other settings, such as inpatients, Emergency Department (ED), and surgery, may behave differently from primary care physicians when managing EHR workload. Physicians in an inpatient and'

In [21]:
[len(enc.encode(c)) for c in chunks[:5]]

[1000, 1000, 1000, 1000, 1000]

In [22]:
chunks_data = []

for idx, row in df4.iterrows():
    article_id = idx
    text = row['text']
    
    chunks = chunk_text_by_tokens(text, max_tokens=1000)
    
    for chunk_id, chunk_text in enumerate(chunks):
        chunks_data.append({
            'article_id': article_id,
            'chunk_id': chunk_id,
            'chunk_text': chunk_text,
            'n_tokens': len(enc.encode(chunk_text))
        })


df_chunks = pd.DataFrame(chunks_data)

In [23]:
df_chunks.head()

,article_id,chunk_id,chunk_text,n_tokens
0,0,0,"## FROM SOVEREIGNTY TO EXTRATERRITORIAL CONSCIOUSNESS Since 1997, the concept of extraterritoriality has been configured in the political tension between Hong Kong and Beijing. From the perspectiv...",1000
1,0,1,"borrowed in 1895 by the Qing officials to redefine the extraterritorial clause in the Treaty of Shimonoseki as a colonial privilege (Cassel 2012: 85-114). During the Republican period (1911-49), ...",1000
2,0,2,"themselves to Mainland's legal regulation or a Mainland production that enjoys certain extraterritorial privileges. According to Lie Fu (2006: 615-18), Yang (2012: 46-51), and Zhao (2008: 216-19)...",1000
3,0,3,"Kong, and within their respective schizophrenic subjectivities is their instance on the idea of making palpable a sense of extraterritorial affectivity. By engaging the spectators in scenarios wh...",1000
4,0,4,"a series of surveillance cameras, from the perspective of a moving car, followed by a repetition of Timmy's medium closeup, where Timmy begins to lose control of the steering wheel. Then, the fil...",1000


In [24]:
df_chunks['n_tokens'].describe()

count    8917.000000
mean      942.155994
std       190.684301
min         1.000000
25%      1000.000000
50%      1000.000000
75%      1000.000000
max      1000.000000
Name: n_tokens, dtype: float64

In [25]:
df_chunks.groupby('article_id').size().describe()

count    1000.000000
mean        8.917000
std         4.636551
min         2.000000
25%         6.000000
50%         8.000000
75%        11.000000
max        58.000000
dtype: float64

In [26]:
df_chunks.to_parquet(
    "../data/processed/articles_chunks.parquet",
    index=False
)

# 5.0 Summary Chunks

In [27]:
model_name = "facebook/bart-large-cnn"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

sample_text = df_chunks.loc[0, "chunk_text"]

inputs = tokenizer(
    sample_text,
    return_tensors="pt",
    truncation=True,
    max_length=1024
)

with torch.no_grad():
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=150,
        min_length=60,
        do_sample=False
    )

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
summary


/home/tiago/.pyenv/versions/llm-docs/lib/python3.10/site-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


"Since 1997, the concept of extraterritoriality has been configured in the political tension between Hong Kong and Beijing. From Beijing's perspective, the Hong Kong legislature is obliged to pass laws that will ascertain the larger political will that the Party represents. But for the residents in Hong Kong, such laws infringe their civil rights and violate the SAR's rights to make its own legal decisions."

In [ ]:
df_chunks['summary'] = None

for i in tqdm(range(len(df_chunks))):
    
    text = df_chunks.loc[i, 'chunk_text']
    
    try:
        summary = summarize_text(text)
        df_chunks.loc[i, 'summary'] = summary
        
    except Exception as e:
        print(f"Erro na linha {i}: {e}")
        df_chunks.loc[i, 'summary'] = None

    # checkpoint
    if i % 50 == 0:
        df_chunks.to_parquet(
            "../data/processed/articles_chunks.parquet",
            index=False
        )